In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from keras.layers import Dense,Lambda
from src.ops import *

Using TensorFlow backend.


In [16]:
class VGAN(object):
    
    def __init__(self,batch_size,z_dim):
        self.batch_size = batch_size
        self.z_dim = z_dim
        self.input_height = 28
        self.input_width = 28
        self.c_dim = 1
    
    def encoder(self,x, is_training=True, reuse=False):
        with tf.variable_scope("encoder", reuse=reuse):
            net = tf.nn.relu(bn(linear(x, 3000, scope='e_fc1')))
            net = tf.nn.relu(bn(linear(net, 2000, scope='e_fc2')))
            e_out = tf.nn.relu(bn(linear(net, self.z_dim+self.y_dim, scope='e_fc3')))
            
            z_out, y_out = Lambda(lambda merge,z_dim:[merge[:,:z_dim],merge[:,z_dim:]],
                                  arguments={'z_dim':self.z_dim})(e_out)
            
            return z_out,y_out
    
    def generator(self, z, y, is_training=True, reuse=False):
        with tf.variable_scope("generator", reuse=reuse):

            # merge z and code
            z = concat([z, y], 1)

            net = tf.nn.relu(bn(linear(z, 1024, scope='g_fc1'), is_training=is_training, scope='g_bn1'))
            net = tf.nn.relu(bn(linear(net, 128 * 7 * 7, scope='g_fc2'), is_training=is_training, scope='g_bn2'))
            net = tf.reshape(net, [self.batch_size, 7, 7, 128])
            net = tf.nn.relu(
                bn(deconv2d(net, [self.batch_size, 14, 14, 64], 4, 4, 2, 2, name='g_dc3'), is_training=is_training,
                   scope='g_bn3'))

            out = tf.nn.sigmoid(deconv2d(net, [self.batch_size, 28, 28, 1], 4, 4, 2, 2, name='g_dc4'))

            return out
        
    def build_model(self):
        
        image_dims = [self.input_height*self.input_width, self.c_dim]

        bs = self.batch_size
        
        """ Graph Input """
        # images
        self.inputs = tf.placeholder(tf.float32, [bs] + image_dims, name='real_images')

        # labels
#         self.y = tf.placeholder(tf.float32, [bs, self.y_dim], name='y')
        E = self.encoder(self.inputs, is_training=True, reuse=True)

        # noises
        self.z = tf.placeholder(tf.float32, [bs, self.z_dim], name='z')
        
        G = self.generator(self.z, self.y, is_training=True, reuse=False)


In [17]:
mod = VGAN(batch_size=64,z_dim=10)

In [18]:
mod.build_model()

ValueError: Trying to share variable encoder/e_fc1/Matrix, but specified shape (784, 3000) and found shape (28, 3000).